In [4]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.config import cities,PG1_CONFIG

import psycopg2

conn = psycopg2.connect(
            dbname=PG1_CONFIG['database'],
            user=PG1_CONFIG['user'],
            password=PG1_CONFIG['password'],
            host=PG1_CONFIG['host'],
            port=PG1_CONFIG['port']
        )
cursor = conn.cursor()

# Insert countries and cities
for city in cities:
    country_name = city["country"]
    city_name = city["city"]
    lat = city["lat"]
    lon = city["lon"]

    # Insert country if not exists
    cursor.execute("""
        INSERT INTO country (CountryName)
        VALUES (%s)
        ON CONFLICT (CountryName) DO NOTHING
        RETURNING IdCountry;
    """, (country_name,))
    country_id = cursor.fetchone()
    if country_id is None:
        cursor.execute("SELECT IdCountry FROM country WHERE CountryName = %s;", (country_name,))
        country_id = cursor.fetchone()[0]
    else:
        country_id = country_id[0]

    # Insert city
    cursor.execute("""
        INSERT INTO city (CityName, IdCountry, Lon, Lat)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (CityName, IdCountry) DO NOTHING;
    """, (city_name, country_id, lon, lat))

# Commit the transaction
conn.commit()

InvalidColumnReference: there is no unique or exclusion constraint matching the ON CONFLICT specification
